In [10]:
import json
import base64
import boto3
from botocore.exceptions import ClientError
import http.client
import psycopg2
import sqlalchemy
import os
import pandas as pd
import numpy as np
import math
import time
import re
# import snowflake.connector
from datetime import datetime
import requests
import uuid

dynamodb = boto3.resource('dynamodb')
dynamodb_table = dynamodb.Table('db_trigger')

# def get_Token():

#           maxabconfig = json.loads(get_secret("prod/maxabconfig/egypt"))
#           os.environ["maxabconfig_user"]= maxabconfig["username"]
#           os.environ["maxabconfig_password"]= maxabconfig["password"]
#           os.environ["maxabconfig_url"]= maxabconfig["url"]

#           body = {
#             "grant_type": "password",
#             "client_id": "admin-portal",
#             "username": os.environ["maxabconfig_user"],
#             "password": os.environ["maxabconfig_password"] }
#           header = { 'content-type': "application/x-www-form-urlencoded" }

#           resp= requests.post(os.environ["maxabconfig_url"], body,headers=header)
#           return resp.json()

# #create a cashin using the cashin portal API 
# def create_cashin(body):
#         base_url = "https://api.maxab.info/emoney/api/integration/v1/cash-ins"
#         access_token = str(get_Token()["access_token"])

#         headers = {
#                 'Authorization': 'Bearer ' + access_token,
#                 'Content-Type': 'application/json'}

#         response = requests.post(base_url, json=body, headers=headers)

#         print(response.status_code)
#         print(response.json())
#         # print(response['cashInId'])
#         # return response['cashInId']
#         return response 



# #staging credentials - will be changed after testing 
# token_URL = "https://sso.maxab.org/auth/realms/maxab/protocol/openid-connect/token"
# def get_Token():   

#           body = {
#             "grant_type": "password",
#             "client_id": "admin-portal",
#             "username": "maxabconfig@maxab.io",
#             "password": "2#g&9C2K9Q855^9ayM"

#         }
#           header = { 'content-type': "application/x-www-form-urlencoded" }

#           resp= requests.post(token_URL, body,headers=header)
#           return resp.json()

# #create a cashin using the cashin portal API 
# def create_cashin(body): 
#         base_url = "https://emoney-staging.maxdev.org/emoney/api/integration/v1/cash-ins"
#         access_token = str(get_Token()["access_token"])

#         headers = {
#                 'Authorization': 'Bearer ' + access_token,
#                 'Content-Type': 'application/json'}

#         response = requests.post(base_url, json=body, headers=headers)

#         print(response.status_code)
#         print(response.json())
#         # print(response['cashInId'])
#         # return response['cashInId']
#         return response

# get secret function:
def get_secret(secret_name):
        region_name = "us-east-1"

        # Create a Secrets Manager client
        session = boto3.session.Session()
        client = session.client(
            service_name='secretsmanager',
            region_name=region_name
        )

        try:
            get_secret_value_response = client.get_secret_value(
                SecretId=secret_name
            )
        except ClientError as e:
            if e.response['Error']['Code'] == 'DecryptionFailureException':
                raise e
            elif e.response['Error']['Code'] == 'InternalServiceErrorException':
                raise e
            elif e.response['Error']['Code'] == 'InvalidParameterException':
                raise e
            elif e.response['Error']['Code'] == 'InvalidRequestException':
                raise e
            elif e.response['Error']['Code'] == 'ResourceNotFoundException':
                raise e
        else:
            if 'SecretString' in get_secret_value_response:
                return get_secret_value_response['SecretString']
            else:
                return base64.b64decode(get_secret_value_response['SecretBinary'])

In [5]:
def lambda_handler(event):
    
        # connect to the dwh db:
        dwh_reader_secret_new = json.loads(get_secret("prod/db/datawarehouse/metabase"))
        dwh_writer_secret_new = json.loads(get_secret("prod/db/datawarehouse/sagemaker"))
        
        # get secrets: -dwh reader
        host_r=dwh_reader_secret_new["host"]
        database_r=dwh_reader_secret_new["dbname"]
        user_r=dwh_reader_secret_new["username"]
        password_r=dwh_reader_secret_new["password"]

        # get secrets: -dwh writer
        host_w=dwh_writer_secret_new["host"]
        database_w=dwh_writer_secret_new["dbname"]
        user_w=dwh_writer_secret_new["username"]
        password_w=dwh_writer_secret_new["password"]

        conn_r = psycopg2.connect(host=host_r, database=database_r, user=user_r, password=password_r)
        print("Successfully connected to reader DB")

        conn_w = psycopg2.connect(host=host_w, database=database_w, user=user_w, password=password_w)
        print("Successfully connected to writer DB")

        #Create a dictionary of the terminal IDs and the linked main system IDs 
        qr_acceptance_profiles = {
        '12791709' : 391593
        # 86910448,
        # 64820792,
        # 11867402,
        # 29247492,
        # 50985068

        }
        
        valid_records_list = []
        for record in event['Records']:
                data = record['dynamodb']['NewImage']
                print(data) 

                #extract needed variables 
                acceptance_id = list(data['id'].values())[0]
                created_at = list(data['created_at'].values())[0]
                created_at = datetime.strptime(created_at, "%Y-%m-%dT%H:%M:%S.%fZ")
                acceptance_created_at = created_at.date()
                terminal_id = list(data['terminal_id'].values())[0]
                #convert the amount to LE 
                amount = int(list(data['amount'].values())[0])/100
                
                #get the main system id linked to the terminal id 
                main_system_id = qr_acceptance_profiles.get(terminal_id,None)
                
                if main_system_id is None: 
                    print('terminal id is not a QR code id') 
                    continue 
                    
                print(acceptance_id, created_at, acceptance_created_at, terminal_id, amount, main_system_id)
                
                #check if the request has already been settled in the settlements table to avoid sending the same cashin twice 
                get_cashin_id_sql = '''
                select cashin_id
                from fintech.qr_acceptance_settlement
                where terminal_id = {} and acceptance_id = {}
                '''.format(terminal_id, acceptance_id)
                
#                 get_cashin_id_sql_result = pd.read_sql(get_cashin_id_sql, conn_r)
                
#                 if not get_cashin_id_sql_result.empty:
#                     print(f"The cashin request with id = {get_cashin_id_sql_result['cashin_id'][0]} has already been settled!")
#                     continue 
                
#                 #insert the new request into table cashins 
#                 myuuid = uuid.uuid4()

#                 body ={
#                       "mainSystemId": main_system_id,
#                       "nonce": "{}".format(str(myuuid)),
#                       "initialAmount": amount,
#                       "tagEn": "QR Acceptance test",
#                       "tagAr": "ايداع بالرمز",
#                       "paymentMethod": "user",
#                       "submittedBy": "e-money",
#                       "cashInMode": "instant",
#                       "initialDate": "{}".format(acceptance_created_at)
#                     }
                                                                                              
#                 try:                                                                                 
#                     response = create_cashin(body)
#                     cashin_id = response.json()['cashInId']
#                 except Exception as e:      
#                     print("error in cashin portal API:{}".format(e))
#                     continue 
                
#                 valid_records_list.append([acceptance_id, terminal_id, cashin_id])
                time.sleep(5)                                                                             
                                                                          
# #insert the values into the settlement table
#         if valid_records_list:
#                 print(valid_records_list)
#                 valid_records_list_df = pd.DataFrame(valid_records_list)
#                 valid_records_list_df.columns = ['acceptance_id', 'terminal_id', 'cashin_id']
#                 valid_records_list_df = valid_records_list_df.reset_index(drop=True)
#                 print(valid_records_list_df)
#                 engine_w = sqlalchemy.create_engine(f"postgresql+psycopg2://{user_w}:{password_w}@{host_w}/{database_w}")
#                 print(bool(engine_w))

#                 with engine_w.connect() as conn:
#                     print("start writing into qr_acceptance_settlement table")
#                     valid_records_list_df.to_sql(name='qr_acceptance_settlement', schema='fintech', con=conn, if_exists='append', index=False, chunksize=1000, method='multi')
#                     print("written into qr_acceptance_settlement table successfully")
#         else:
#             print('no records found')
    
     # close the connection:
        conn_w.close()
        conn_r.close()

        


In [6]:
lambda_handler(
    {
   "Records":[
      {
         "eventID":"dcc995dc7cc79c12f3db5d2fe3fe3e9e",
         "eventName":"INSERT",
         "eventVersion":"1.1",
         "eventSource":"aws:dynamodb",
         "awsRegion":"us-east-1",
         "dynamodb":{
            "ApproximateCreationDateTime":1719227295.0,
            "Keys":{
               "id":{
                  "N":"614"
               }
            },
            "NewImage":{
               "amount":{
                  "S":"100"
               },
               "paid_through":{
                  "S":"Tahweel"
               },
               "action_code":{
                  "S":"00"
               },
               "created_at":{
                  "S":"2024-06-24T11:08:15.611864Z"
               },
               "merchant_id":{
                  "S":"10069060210"
               },
               "message":{
                  "S":"Approved"
               },
               "authorization_date_time":{
                  "S":"20240624140812"
               },
               "payer_account":{
                  "S":"01092260906"
               },
               "additional_fees":{
                  "S":"0"
               },
               "updated_at":{
                  "S":"2024-06-24T11:08:15.611864Z"
               },
               "system_reference":{
                  "S":"33769125"
               },
               "txn_type":{
                  "S":"6"
               },
               "date_time_local_trxn":{
                  "S":"20240624140815"
               },
               "network_reference":{
                  "S":"f778ab47-da05-4a2f-aa06-e09be6c43e53"
               },
               "secure_hash":{
                  "S":"B63C1911E84A154DD601606ED27839B7123544CCDFB4866FF4515E85E925B59D"
               },
               "currency":{
                  "S":"818"
               },
               "tip":{
                  "S":"0"
               },
               "id":{
                  "N":"201"
               },
               "payer_name":{
                  "S":"Mohamed A Reyad"
               },
               "terminal_id":{
                  "S":"127909"
               },
               "status":{
                  "S":"Success"
               }
            },
            "SequenceNumber":"45500000000090750472371",
            "SizeBytes":506,
            "StreamViewType":"NEW_AND_OLD_IMAGES"
         },
         "eventSourceARN":"arn:aws:dynamodb:us-east-1:876425898567:table/upg_failed_requests/stream/2024-06-24T11:00:55.726"
      }
   ]
}
)

Successfully connected to reader DB
Successfully connected to writer DB
{'amount': {'S': '100'}, 'paid_through': {'S': 'Tahweel'}, 'action_code': {'S': '00'}, 'created_at': {'S': '2024-06-24T11:08:15.611864Z'}, 'merchant_id': {'S': '10069060210'}, 'message': {'S': 'Approved'}, 'authorization_date_time': {'S': '20240624140812'}, 'payer_account': {'S': '01092260906'}, 'additional_fees': {'S': '0'}, 'updated_at': {'S': '2024-06-24T11:08:15.611864Z'}, 'system_reference': {'S': '33769125'}, 'txn_type': {'S': '6'}, 'date_time_local_trxn': {'S': '20240624140815'}, 'network_reference': {'S': 'f778ab47-da05-4a2f-aa06-e09be6c43e53'}, 'secure_hash': {'S': 'B63C1911E84A154DD601606ED27839B7123544CCDFB4866FF4515E85E925B59D'}, 'currency': {'S': '818'}, 'tip': {'S': '0'}, 'id': {'N': '201'}, 'payer_name': {'S': 'Mohamed A Reyad'}, 'terminal_id': {'S': '127909'}, 'status': {'S': 'Success'}}
terminal id is not a QR code id


In [43]:
import requests
import uuid

token_URL = "https://sso.maxab.org/auth/realms/maxab/protocol/openid-connect/token"
def get_Token(): 
  body = {
    "grant_type": "password",
    "client_id": "admin-portal",
    "username": "maxabconfig@maxab.io",
    "password": "2#g&9C2K9Q855^9ayM"

}
  header = { 'content-type': "application/x-www-form-urlencoded" }

  resp= requests.post(token_URL, body,headers=header)
  return resp.json()



def create_cashin(body, base, access_token):
    print(body)
    headers = {
        'Authorization': 'Bearer ' + access_token,
        'Content-Type': 'application/json'
    }

    response = requests.post(base, json=body, headers=headers)

    print(response)
    return response

# Example usage:
base_url = "https://emoney-staging.maxdev.org/emoney/api/integration/v1/cash-ins"
body ={
  "mainSystemId": 94474,
  "nonce": "333a4c3f-b76d-454f-9f1c-6cde69486659",
  "initialAmount": 1000,
  "tagEn": "qr_acceptance_test3",
  "tagAr": "كيو ر تست",
  "paymentMethod": "user",
  "submittedBy": "e-money",
  "cashInMode": "instant",
  "initialDate": "2024-06-26"
}
access_token = str(get_Token()["access_token"])
response = create_cashin(body, base_url, access_token)
print(response.status_code)
print(response.json())

{'mainSystemId': 94474, 'nonce': '333a4c3f-b76d-454f-9f1c-6cde69486659', 'initialAmount': 1000, 'tagEn': 'qr_acceptance_test3', 'tagAr': 'كيو ر تست', 'paymentMethod': 'user', 'submittedBy': 'e-money', 'cashInMode': 'instant', 'initialDate': '2024-06-26'}
<Response [200]>
200
{'cashInId': 1017338}


In [27]:
import json
import base64
import boto3
from botocore.exceptions import ClientError
import http.client
import psycopg2
import sqlalchemy
import os
import pandas as pd
import numpy as np
import math
import time
import re 
# import snowflake.connector
import requests
import uuid
import oauth2client
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint
from df2gspread import df2gspread as d2g
import sys
import requests
import importlib
import os
import psycopg2
from datetime import datetime, timedelta, date
import copy
from tqdm import tqdm
import datetime as dt
from oauth2client.service_account import ServiceAccountCredentials
from pathlib import Path


def get_Token():

          maxabconfig = json.loads(get_secret("prod/maxabconfig/egypt"))
          os.environ["maxabconfig_user"]= maxabconfig["username"]
          os.environ["maxabconfig_password"]= maxabconfig["password"]
          os.environ["maxabconfig_url"]= maxabconfig["url"]

          body = {
            "grant_type": "password",
            "client_id": "admin-portal",
            "username": os.environ["maxabconfig_user"],
            "password": os.environ["maxabconfig_password"] }
          header = { 'content-type': "application/x-www-form-urlencoded" }

          resp= requests.post(os.environ["maxabconfig_url"], body,headers=header)
          return resp.json()
        
        
#create a cashin using the cashin portal API 
def create_cashin(body):
        base_url = "https://api.maxab.info/emoney/api/integration/v1/cash-ins"
        access_token = str(get_Token()["access_token"])

        headers = {
                'Authorization': 'Bearer ' + access_token,
                'Content-Type': 'application/json'}

        response = requests.post(base_url, json=body, headers=headers)

        print(response.status_code)
        print(response.json())
        # print(response['cashInId'])
        # return response['cashInId']
        return response 
    
# get secret function:
def get_secret(secret_name):
        region_name = "us-east-1"

        # Create a Secrets Manager client
        session = boto3.session.Session()
        client = session.client(
            service_name='secretsmanager',
            region_name=region_name
        )

        try:
            get_secret_value_response = client.get_secret_value(
                SecretId=secret_name
            )
        except ClientError as e:
            if e.response['Error']['Code'] == 'DecryptionFailureException':
                raise e
            elif e.response['Error']['Code'] == 'InternalServiceErrorException':
                raise e
            elif e.response['Error']['Code'] == 'InvalidParameterException':
                raise e
            elif e.response['Error']['Code'] == 'InvalidRequestException':
                raise e
            elif e.response['Error']['Code'] == 'ResourceNotFoundException':
                raise e
        else:
            if 'SecretString' in get_secret_value_response:
                return get_secret_value_response['SecretString']
            else:
                return base64.b64decode(get_secret_value_response['SecretBinary'])


def get_from_gsheet(workbook, sheet):
    scope = ["https://spreadsheets.google.com/feeds",
         'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file",
         "https://www.googleapis.com/auth/drive"]
    initialize_env()
    creds = ServiceAccountCredentials.from_json_keyfile_name(os.environ["GOOGLE_APPLICATION_CREDENTIALS_SHEETS"], scope)
    client = gspread.authorize(creds)
    try:
        wks = client.open(workbook).worksheet(sheet)
        sheet = pd.DataFrame(wks.get_all_records())
    except:
        print(sheet,'failed to fetch data')
    
    return sheet


#insert the new request into table cashins 
myuuid = uuid.uuid4()

body ={
      "mainSystemId": 391593,
      "nonce": "{}".format(str(myuuid)),
      "initialAmount": 1,
      "tagEn": "test",
      "tagAr": "test",
      "paymentMethod": "user",
      "submittedBy": "e-money",
      "cashInMode": "instant",
      "initialDate": "18/8/2024"
    }


try:
    response = create_cashin(body)
    if response.status_code != 200:
        print("here")
        fallback_list = [391593,2,'2024-8-18',2,10]
        scope = [
        "https://spreadsheets.google.com/feeds",
        'https://www.googleapis.com/auth/spreadsheets',
        "https://www.googleapis.com/auth/drive.file",
        "https://www.googleapis.com/auth/drive"
        ]
        creds = ServiceAccountCredentials.from_json_keyfile_name(os.environ["GOOGLE_APPLICATION_CREDENTIALS_SHEETS"], scope)
        client = gspread.authorize(creds)
        google_sh = client.open('QR Fallback Sheet')
        sheet = google_sh.worksheet('Sheet1')
        # sheet.clear()
        sheet.append_row(fallback_list, value_input_option="USER_ENTERED")
    cashin_id = response.json()['cashInId']
except Exception as e:
    print("error in cashin portal API:{}".format(e))


400
{'code': None, 'message': 'Error parsing JSON', 'details': None, 'violations': None, 'extraData': None}
here
error in cashin portal API:'cashInId'


## LAST VERSION

In [ ]:
pip install gspread_dataframe

In [28]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import os
import pandas as pd
import json
import base64
import boto3
from botocore.exceptions import ClientError
import http.client
import psycopg2
import sqlalchemy
import numpy as np
import math
import time
import re
# import snowflake.connector
from datetime import datetime
import requests
import uuid
from pathlib import Path


def initialize_env():
    json_path_sheets = str(Path.home()) + "/service_account_key_sheets.json"
    sheets_key = get_secret("prod/maxab-sheets")
    f = open(json_path_sheets, "w")
    f.write(sheets_key)
    f.close()
    os.environ["GOOGLE_APPLICATION_CREDENTIALS_SHEETS"] = json_path_sheets


def google_sheets(workbook, sheet, action,cols=[], df=None):
    initialize_env()

    scope = [
        'https://spreadsheets.google.com/feeds',
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive.file',
        'https://www.googleapis.com/auth/drive'
    ]

    creds = ServiceAccountCredentials.from_json_keyfile_name(os.environ["GOOGLE_APPLICATION_CREDENTIALS_SHEETS"] , scope)
    client = gspread.authorize(creds)
    wks = client.open(workbook).worksheet(sheet)

    if action.lower() == 'get':
        if len(cols) > 0:
            sheet = get_as_dataframe(
                wks, 
                parse_dates=True,
                usecols = cols,
                evaluate_formulas=True
            ).dropna(how = 'all')
        elif len(cols) == 0:
            sheet = get_as_dataframe(
                wks,
                parse_dates=True,
                evaluate_formulas=True
            ).dropna(how = 'all')
            
        return sheet
    elif action.lower() == 'overwrite':
        wks.clear()
        set_with_dataframe(wks, df)
        return('data is added to the sheet successfully')
    elif action.lower() == 'append':
        existing_data = get_as_dataframe(wks)
        updated_data = pd.concat([existing_data, df], ignore_index=True)
        set_with_dataframe(wks, updated_data)
        return 'Data is appended to the sheet successfully'
    elif action.lower() == 'clear':
        wks.clear()
        return('sheet is cleared')

In [60]:
qr_acceptance_profiles = google_sheets('QR Acceptance Profiles List','Sheet2','get')
qr_acceptance_profiles[['Terminal_ID', 'Scheme_MID', 'Main_System_ID']] = qr_acceptance_profiles[['Terminal_ID', 'Scheme_MID', 'Main_System_ID']].astype(int)
print(qr_acceptance_profiles.shape)
print(qr_acceptance_profiles.dtypes)
qr_acceptance_profiles.head()
qr_profiles_df = qr_acceptance_profiles[['Terminal_ID', 'Main_System_ID']]
qr_profiles_df.head()

filtered_df = qr_profiles_df[qr_profiles_df['Terminal_ID'] == int('86910448')]

# Check if the result is not empty and retrieve the value
if not filtered_df.empty:
    main_system_id = filtered_df['Main_System_ID'].values[0]
    print(main_system_id)

(50, 5)
Terminal_ID         int64
Scheme_MID          int64
Main_System_ID      int64
Start_Date         object
End_Date          float64
dtype: object
508908


In [63]:

dynamodb = boto3.resource('dynamodb')
dynamodb_table = dynamodb.Table('db_trigger')

def get_Token():

          maxabconfig = json.loads(get_secret("prod/maxabconfig/egypt"))
          os.environ["maxabconfig_user"]= maxabconfig["username"]
          os.environ["maxabconfig_password"]= maxabconfig["password"]
          os.environ["maxabconfig_url"]= maxabconfig["url"]

          body = {
            "grant_type": "password",
            "client_id": "admin-portal",
            "username": os.environ["maxabconfig_user"],
            "password": os.environ["maxabconfig_password"] }
          header = { 'content-type': "application/x-www-form-urlencoded" }

          resp= requests.post(os.environ["maxabconfig_url"], body,headers=header)
          return resp.json()

#create a cashin using the cashin portal API 
def create_cashin(body):
        base_url = "https://api.maxab.info/emoney/api/integration/v1/cash-ins"
        access_token = str(get_Token()["access_token"])

        headers = {
                'Authorization': 'Bearer ' + access_token,
                'Content-Type': 'application/json'}

        response = requests.post(base_url, json=body, headers=headers)

        print(response.status_code)
        print(response.json())
        # print(response['cashInId'])
        # return response['cashInId']
        return response 


# get secret function:
def get_secret(secret_name):
        region_name = "us-east-1"

        # Create a Secrets Manager client
        session = boto3.session.Session()
        client = session.client(
            service_name='secretsmanager',
            region_name=region_name
        )

        try:
            get_secret_value_response = client.get_secret_value(
                SecretId=secret_name
            )
        except ClientError as e:
            if e.response['Error']['Code'] == 'DecryptionFailureException':
                raise e
            elif e.response['Error']['Code'] == 'InternalServiceErrorException':
                raise e
            elif e.response['Error']['Code'] == 'InvalidParameterException':
                raise e
            elif e.response['Error']['Code'] == 'InvalidRequestException':
                raise e
            elif e.response['Error']['Code'] == 'ResourceNotFoundException':
                raise e
        else:
            if 'SecretString' in get_secret_value_response:
                return get_secret_value_response['SecretString']
            else:
                return base64.b64decode(get_secret_value_response['SecretBinary'])

                
def lambda_handler(event):

        # connect to the dwh db:
        dwh_reader_secret_new = json.loads(get_secret("prod/db/datawarehouse/metabase"))
        dwh_writer_secret_new = json.loads(get_secret("prod/db/datawarehouse/sagemaker"))

        # get secrets: -dwh reader
        host_r=dwh_reader_secret_new["host"]
        database_r=dwh_reader_secret_new["dbname"]
        user_r=dwh_reader_secret_new["username"]
        password_r=dwh_reader_secret_new["password"]

        # get secrets: -dwh writer
        host_w=dwh_writer_secret_new["host"]
        database_w=dwh_writer_secret_new["dbname"]
        user_w=dwh_writer_secret_new["username"]
        password_w=dwh_writer_secret_new["password"]

        conn_r = psycopg2.connect(host=host_r, database=database_r, user=user_r, password=password_r)
        print("Successfully connected to reader DB")

        conn_w = psycopg2.connect(host=host_w, database=database_w, user=user_w, password=password_w)
        print("Successfully connected to writer DB")

        #Get the qr profiles list from the sheet 
        qr_acceptanc_profiles = google_sheets('QR Acceptance Profiles List','Sheet2','get')
        qr_acceptance_profiles[['Terminal_ID', 'Scheme_MID', 'Main_System_ID']] = qr_acceptance_profiles[['Terminal_ID', 'Scheme_MID', 'Main_System_ID']].astype(int)
        qr_profiles_df = qr_acceptance_profiles[['Terminal_ID', 'Main_System_ID']]
        
        valid_records_list = []
        for record in event['Records']:
                data = record['dynamodb']['NewImage']
                print(data) 

                #extract needed variables 
                acceptance_id = list(data['id'].values())[0]
                created_at = list(data['created_at'].values())[0]
                created_at = datetime.strptime(created_at, "%Y-%m-%dT%H:%M:%S.%fZ")
                acceptance_created_at = created_at.date()
                terminal_id = list(data['terminal_id'].values())[0]
                #convert the amount to LE 
                amount = int(list(data['amount'].values())[0])/100
                
                #get the main system id linked to the terminal id 
                #check if its a qr terminal id and skip if its not 
                
                filtered_df = qr_profiles_df[qr_profiles_df['Terminal_ID'] == int(terminal_id)]
                print(filtered_df)
                # Check if the result is not empty and retrieve the value
                if not filtered_df.empty:
                    main_system_id = filtered_df['Main_System_ID'].values[0]
                if main_system_id is None:
                    print('terminal id is not a QR code id')
                    continue

                print(acceptance_id, created_at, acceptance_created_at, terminal_id, amount, main_system_id)
                
                #check if the request has already been settled in the settlements table to avoid sending the same cashin twice 
                get_cashin_id_sql = '''
                select cashin_id
                from fintech.qr_acceptance_settlement
                where terminal_id = {} and acceptance_id = {}
                '''.format(terminal_id, acceptance_id)
                
                get_cashin_id_sql_result = pd.read_sql(get_cashin_id_sql, conn_r)
                
                if not get_cashin_id_sql_result.empty:
                    print(f"The cashin request with id = {get_cashin_id_sql_result['cashin_id'][0]} has already been settled!")
                    continue
                
                #insert the new request into table cashins 
                myuuid = uuid.uuid4()

                body ={
                      "mainSystemId": str(main_system_id),
                      "nonce": "{}".format(str(myuuid)),
                      "initialAmount": amount,
                      "tagEn": "QR Acceptance",
                      "tagAr": "معاملة ايداع بالرمز",
                      "paymentMethod": "user",
                      "submittedBy": "e-money",
                      "cashInMode": "instant",
                      "initialDate": "{}".format(acceptance_created_at)
                    }

                try:
                    response = create_cashin(body)
                    cashin_id = response.json()['cashInId']
                except Exception as e:      
                    print("error in cashin portal API:{}".format(e))
                    time.sleep(5)
                    try:
                        response = create_cashin(body)
                        cashin_id = response.json()['cashInId']
                    except Exception as ee: 
                        print("error again in cashin portal API:{}".format(e))
                        continue

                valid_records_list.append([acceptance_id, terminal_id, cashin_id])
                time.sleep(2)

#insert the values into the settlement table
        if valid_records_list:
                print(valid_records_list)
                valid_records_list_df = pd.DataFrame(valid_records_list)
                valid_records_list_df.columns = ['acceptance_id', 'terminal_id', 'cashin_id']
                valid_records_list_df = valid_records_list_df.reset_index(drop=True)
                print(valid_records_list_df)
                engine_w = sqlalchemy.create_engine(f"postgresql+psycopg2://{user_w}:{password_w}@{host_w}/{database_w}")
                print(bool(engine_w))

                with engine_w.connect() as conn:
                    print("start writing into qr_acceptance_settlement table")
                    valid_records_list_df.to_sql(name='qr_acceptance_settlement', schema='fintech', con=conn, if_exists='append', index=False, chunksize=1000, method='multi')
                    print("written into qr_acceptance_settlement table successfully")
        else:
            print('no records found')
    
     # close the connection:
        conn_w.close()
        conn_r.close()

        return {
            'statusCode': 200,
            'body': json.dumps('Hello from Lambda!')
        }


In [64]:
lambda_handler(
    {
   "Records":[
      {
         "eventID":"dcc995dc7cc79c12f3db5d2fe3fe3e9e",
         "eventName":"INSERT",
         "eventVersion":"1.1",
         "eventSource":"aws:dynamodb",
         "awsRegion":"us-east-1",
         "dynamodb":{
            "ApproximateCreationDateTime":1719227295.0,
            "Keys":{
               "id":{
                  "N":"614"
               }
            },
            "NewImage":{
               "amount":{
                  "S":"100"
               },
               "paid_through":{
                  "S":"Tahweel"
               },
               "action_code":{
                  "S":"00"
               },
               "created_at":{
                  "S":"2024-09-16T16:08:15.611864Z"
               },
               "merchant_id":{
                  "S":"10069060210"
               },
               "message":{
                  "S":"Approved"
               },
               "authorization_date_time":{
                  "S":"20240916160815"
               },
               "payer_account":{
                  "S":"01092260906"
               },
               "additional_fees":{
                  "S":"0"
               },
               "updated_at":{
                  "S":"2024-09-16T16:08:15.611864Z"
               },
               "system_reference":{
                  "S":"33769125"
               },
               "txn_type":{
                  "S":"6"
               },
               "date_time_local_trxn":{
                  "S":"20240916160815"
               },
               "network_reference":{
                  "S":"f778ab47-da05-4a2f-aa06-e09be6c43e53"
               },
               "secure_hash":{
                  "S":"B63C1911E84A154DD601606ED27839B7123544CCDFB4866FF4515E85E925B59D"
               },
               "currency":{
                  "S":"818"
               },
               "tip":{
                  "S":"0"
               },
               "id":{
                  "N":"201"
               },
               "payer_name":{
                  "S":"Nadine Test"
               },
               "terminal_id":{
                  "S":"29247492"
               },
               "status":{
                  "S":"Success"
               }
            },
            "SequenceNumber":"45500000000090750472371",
            "SizeBytes":506,
            "StreamViewType":"NEW_AND_OLD_IMAGES"
         },
         "eventSourceARN":"arn:aws:dynamodb:us-east-1:876425898567:table/upg_failed_requests/stream/2024-06-24T11:00:55.726"
      }
   ]
}
)

Successfully connected to reader DB
Successfully connected to writer DB
{'amount': {'S': '100'}, 'paid_through': {'S': 'Tahweel'}, 'action_code': {'S': '00'}, 'created_at': {'S': '2024-09-16T16:08:15.611864Z'}, 'merchant_id': {'S': '10069060210'}, 'message': {'S': 'Approved'}, 'authorization_date_time': {'S': '20240916160815'}, 'payer_account': {'S': '01092260906'}, 'additional_fees': {'S': '0'}, 'updated_at': {'S': '2024-09-16T16:08:15.611864Z'}, 'system_reference': {'S': '33769125'}, 'txn_type': {'S': '6'}, 'date_time_local_trxn': {'S': '20240916160815'}, 'network_reference': {'S': 'f778ab47-da05-4a2f-aa06-e09be6c43e53'}, 'secure_hash': {'S': 'B63C1911E84A154DD601606ED27839B7123544CCDFB4866FF4515E85E925B59D'}, 'currency': {'S': '818'}, 'tip': {'S': '0'}, 'id': {'N': '201'}, 'payer_name': {'S': 'Nadine Test'}, 'terminal_id': {'S': '29247492'}, 'status': {'S': 'Success'}}
   Terminal_ID  Main_System_ID
2     29247492          391593
201 2024-09-16 16:08:15.611864 2024-09-16 29247492 1.

/tmp/ipykernel_9045/216051834.py:136: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  get_cashin_id_sql_result = pd.read_sql(get_cashin_id_sql, conn_r)


200
{'cashInId': 9397373}
[['201', '29247492', 9397373]]
  acceptance_id terminal_id  cashin_id
0           201    29247492    9397373
True
start writing into qr_acceptance_settlement table
written into qr_acceptance_settlement table successfully


{'statusCode': 200, 'body': '"Hello from Lambda!"'}